In [1]:
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
import matplotlib.pyplot as plt
import cv2
import numpy as np      
from time import sleep

base = BaseOverlay("base.bit")
print("overlay init")

overlay init


In [2]:
# 硬件模块
sw0 = base.switches[0]
sw1 = base.switches[1]
rgbled_position = [4,5]
print("buttons init")

buttons init


In [3]:
# 语音模块
pAudio = base.audio 
pAudio.select_microphone()

def audio_play(str) :
    if str == 'known':
        pAudio.load("/home/xilinx/jupyter_notebooks/face/audio/known.wav")
        pAudio.play()
    elif str == 'unknown':
        pAudio.load("/home/xilinx/jupyter_notebooks/face/audio/unknown.wav")
        pAudio.play()

print("audio init")

audio init


In [4]:
# 面容识别模块
recognizer = cv2.face.createLBPHFaceRecognizer()
recognizer.load(
    '/home/xilinx/jupyter_notebooks/face/'
    'trainner.yml')
face_cascade = cv2.CascadeClassifier(
    '/home/xilinx/jupyter_notebooks/face/'
    'haarcascade_frontalface_alt.xml')
# 变量定义
idnum = 0
p = [0,0]
count = 0
count1 = 0
# 录入用户编号
names = ['初始', 'zyb', 'jwj', 'user2', 'user3']
print("face recognition init")

face recognition success


In [5]:
# 姿态检测模块
fg = cv2.createBackgroundSubtractorMOG2()  # 实例化高斯混合模型
print("gesture init")

In [6]:
# HDMI & 视频模块
Mode = VideoMode(640,480,24)
hdmi_out = base.video.hdmi_out
hdmi_out.configure(Mode,PIXEL_BGR)
hdmi_out.start()

frame_out_w = 640
frame_out_h = 480

frame_in_w = 640
frame_in_h = 480

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h)
minW = 0.1 * cap.get(3)
minH = 0.1 * cap.get(4)
print("capture device is open: " + str(cap.isOpened()))

capture device is open: True


In [7]:
cap.read()
cap.read()
cap.read()
cap.read()
cap.read()
print("begin")

while True:
    ret, frame = cap.read()
    
    if sw0.read() == 0:
         # 识别人脸
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray,
                                              scaleFactor=1.2,
                                              minNeighbors=5,
                                              minSize=(int(minW), int(minH)))
        if faces is not None:

            # 进行校验
            for (x, y, w, h) in faces:
                if count1 < 10:
                    count1 = count1 + 1
                count = count + 1
                idnum, confidence = recognizer.predict(gray[y:y + h, x:x + w])
                print(idnum, confidence)
    #             plt.imshow(frame)
    #             plt.show()
                if confidence < 40:
    #                 print("jwj")
                    p[0] = p[0] + 1
                else:
                    p[1] = p[1] + 1

        if count1 < 10:
            continue
    #     print("we start to detect the face")
        if count >= 5:
            if p[0] == 0 and p[1] == 0:
                continue
            elif p[0] != 0:
                print("known")
                base.rgbleds[4].write(1)
                base.rgbleds[5].write(1)
                audio_play('known') 
            else:
                print("unknown")
                base.rgbleds[4].write(6)
                base.rgbleds[5].write(6)
                audio_play('unknown') 
            p[0] = 0
            p[1] = 0
            count = 0
    if sw0.read() == 1:
        blurred = cv2.GaussianBlur(frame, (3, 3), 0)  # 高斯滤波
        gray = cv2.cvtColor(blurred, cv2.COLOR_RGB2GRAY)  # 灰度图
        xgrad = cv2.Sobel(gray, cv2.CV_16SC1, 1, 0)  # x方向梯度
        ygrad = cv2.Sobel(gray, cv2.CV_16SC1, 0, 1)  # y方向梯度
        edge_output = cv2.Canny(xgrad, ygrad, 50, 150)  # canny 边缘检测
        fgmask = fg.apply(edge_output)  # 使用高斯混合模型，背景减除
        
        # 闭运算
        hline = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 4),
                                          (-1, -1))  # 定义结构元素，卷积核
        vline = cv2.getStructuringElement(cv2.MORPH_RECT, (4, 1), (-1, -1))
        result = cv2.morphologyEx(fgmask, cv2.MORPH_CLOSE, hline)  # 水平方向
        result = cv2.morphologyEx(result, cv2.MORPH_CLOSE, vline)  # 垂直方向
        dilateim = cv2.dilate(result,
                              cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (4, 4)),
                              iterations=1)  # 使边缘膨胀
        frame, contours, hierarchy  = cv2.findContours(dilateim, cv2.RETR_EXTERNAL,
                                          cv2.CHAIN_APPROX_NONE)  # 查找轮廓

        for c in contours:
            if cv2.contourArea(c) > 10000:
                (x, y, w, h) = cv2.boundingRect(c)
                scale = w / h

        # 根据比例判断， 首先人不能离摄像头太近，其次可以打印scale来调参
        if scale > 2:
            print("摔倒了")
#             加语音播放“危险”
        else:
            print("行走中")
        
    cv2.waitKey(1)      
        
    # switches开关
    if sw1.read() == 1:
        print("close window")
        break
        
    

    

base.rgbleds[4].write(0)
base.rgbleds[5].write(0)
cv2.waitKey(1)
cap.release()
print("video over")

begin
2 73.28347671646438
2 64.0548472557951
2 72.6210948146706
2 65.49762063569257
2 61.04111606430224
2 65.3873692635379
2 54.05675705720954
2 57.4931296647121
2 55.771595566111266
2 56.87349237910585
unknown
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
摔倒了
close window
video over


#### HDMI 输出部分
outframe = hdmi_out.newframe()
outframe[:] = frame
hdmi_out.writeframe(outframe)